# NEAT Training
***
The following notebook is used to train the NEAT evolutionary algorithm for Tetris.
A game instance is imported and used as a genome for training the model.

### Imports
***
All the necessary imports to train the model

In [35]:
import os
import numpy as np
import neat
import pickle # Used to save the model
import multiprocessing
import tetris
import graphics

### Config File
***
This cell modify the config txt file according to the parameter of the NEAT algorithm

In [36]:
### NEAT CONFIGURATION ###
pop_size = 10
fitness_threshold = 1000
num_inputs = 2 + 22*12
num_outputs = 2 # 2 possible actions : number of rotate and index of columns to place the piece

with open("config.txt", "r") as f:
    pop_index, threshold_index, inputs_index, outputs_index = 0, 0, 0, 0
    lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].startswith("pop_size"):
            pop_index = i
        elif lines[i].startswith("fitness_threshold"):
            threshold_index = i
        elif lines[i].startswith("num_inputs"):
            inputs_index = i
        elif lines[i].startswith("num_outputs"):
            outputs_index = i

with open("config.txt", "w") as f:
    lines[pop_index] = f"pop_size              = {pop_size}\n"
    lines[threshold_index] = f"fitness_threshold     = {fitness_threshold}\n"
    lines[inputs_index] = f"num_inputs              = {num_inputs}\n"
    lines[outputs_index] = f"num_outputs             = {num_outputs}\n"
    f.writelines(lines)

    

### Evaluation function
***
Evaluation function that will be used in the thread pool

In [37]:
def convert_command(commands, num_columns):
    command_1, command_2 = commands

    # Converts to number of rotations
    if command_1 <= -0.5:
        command_1 = 0
    elif -0.5 < command_1 <= 0:
        command_1 = 1
    elif 0 < command_1 <= 0.5:
        command_1 = 2
    else:
        command_1 = 3

    # Converts to index of column
    command_2 += 1
    command_2 *= num_columns // 2 - 1
    command_2 = int(command_2)

    return [command_1, command_2]

def flatten_matrix(matrix):
    """Flattens a matrix into a list

    Args:
        matrix (list): matrix to flatten

    Returns:
        list: flattened matrix
    """
    return [item for sublist in matrix for item in sublist]

def eval_genomes(genomes, config):
    """Creates the eval function that will be used in a thread pool for each genome

    Args:
        genomes (genome): genome object
        config (config): config object
        
        returns : fitness of the genome
    """
    tet = []
    ge = []
    nets = []

    for _id, genome in genomes:
        tet.append(tetris.Tetris())
        ge.append(genome)
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        nets.append(net)
        genome.fitness = 0

    # Runs the game until it ends
    while True:
        # game_state = t.board  # Gets the current state of the game
        # current_block = t.current_piece
        # next_block = t.next_piece
        # commands = net.activate(current_block, next_block, *flatten_matrix(game_state)) # Calculates the output of the network according to the current state
        # commands = convert_command(commands, 12) # Gives the number of rotations and the index of the column to place the piece
        # t.play(*commands)
        # t.next_pos()    # Updates the game state according to the output of the network

        for i in range(len(tet)):
            
            if tet[i].game_over:
                tet.pop(i)
                ge.pop(i)
                nets.pop(i)

            if len(tet) == 0:
                return
                
            game_state = tet[i].board  # Gets the current state of the game
            current_block = tet[i].current_piece
            next_block = tet[i].next_piece
            commands = nets[i].activate((current_block, next_block, *flatten_matrix(game_state)))
            commands = convert_command(commands, 12)
            tet[i].next_pos()
            tet[i].play(*commands)
            ge[i].fitness = tet[i].score
            



### Run function
***
Runs the evaluation function for each genome that are each placed in a different thread pool

In [38]:
def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)
    
    p = neat.Population(config) # Creates the population
    winner = p.run(eval_genomes, pop_size) # Runs the population until the fitness threshold is reached
    with open("winner.pkl", "wb") as f:
        pickle.dump(winner, f)
        f.close()

run("config.txt")

IndexError: index 22 is out of bounds for axis 0 with size 22

### Loads and tests
***
Loads the best genome stored in the .pkl file and tests it on a test game

In [ ]:
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         "config.txt")
genome = pickle.load(open('winner.pkl', 'rb'))
net = neat.nn.FeedForwardNetworks.create(genome, config)
# Tests the best genome on a test game
t = tetris.Tetris()
graphic = graphics.Graphic(300, (64, 201, 255), (232, 28, 255), (255, 255, 255), t.board)
while not t.game_over:
    state = t.board
    current_block = t.current_piece
    next_block = t.next_piece
    commands = net.activate((current_block, next_block, *flatten_matrix(state)))
    commands = convert_command(commands, 12)
    t.play(*commands)
    t.next_pos()
    graphic.draw(t.board)
    
